In [ ]:
from matplotlib import cm, pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_score
import time
from sklearn import svm
from sklearn.model_selection import train_test_split


def load_dataset(partition):
    X,y = fetch_openml('mnist_784', version=1, return_X_y=True)
    y = y.astype(int)[:int(len(X)*partition//1)]
    X = X/255
    X=X[:int(len(X)*partition//1)]
    return X,y

def tune_parameter(parname, parvalues ,cf ,k_fold,X,y):
    acc=[]
    for i in range(len(parvalues)):
        cf.set_params(**{parname:parvalues[i]})
        acc.append(cross_val_score(cf, X, y, cv=k_fold,scoring="accuracy"))
    return acc

In [ ]:
X,y=load_dataset(.5)
X1=X.drop(X.columns[np.std(X)==0.0],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X1,y,test_size=.20)

c:\Users\schio\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
C=[0.1 ,1 ]
cl = svm.SVC(kernel='linear', C=1)
start = time.time()
t_acc=tune_parameter("C",t_values,cl,10,X_train,y_train)
end = time.time()
print("10-way cross validation mean time "+str(int((end - start)/len(t_values)))+"s")

0.9341428571428572
